## 딥러닝용 더미 데이터 생성 자동화 프로젝트


In [1]:

# 주피터 노트북용 디버그
from IPython.display import display


class Logger(object):
    """
    디버그 로그 관리
    """

    def __init__(self):
        # 디버깅용 출력 여부 설정
        self.is_debug = True

    def is_debug(self):
        return self.is_debug

    def set_debug(self, is_debug=True):
        self.is_debug = is_debug

    def d(self, arg):
        if self.is_debug:
            display(arg)
            

In [2]:

"""
주어진 csv 파일을 본떠서 더미 csv 파일을 만드는 소스를 생성하는 코드이다
"""

__author__ = 'Hyungkoo Kim'
__copyright__ = 'Copyright 2020, CHONNAM NATIONAL UNIVERSITY'
__date__ = '2020/09/11'


import numpy as np
import pandas as pd


In [3]:

import os
from datetime import datetime

import numpy as np
import pandas as pd


class DLDummyGenerator(object):
    """
    딥러닝용 더미 데이터 생성 자동화
    """
    pass


In [4]:

class DLDummyGenerator(DLDummyGenerator):
    
    def __init__(self, csv_file_name, gen_row_len=5, unique_field_count=1000, date_fields=[[]], logger=Logger()):
        """
        :param csv_file_name (str): 생성할 소스 파일의 기준이 되는 csv 데이터 파일
        :param gen_row_len (int):   생성할 소스에서 랜덤 생성되는 더미 데이터의 행 길이
        :param unique_field_count (int):    유니크한 문자열 (예를 들어서 코드 값) 필드로 뽑아낼 기준의 크기.
        :param date_fields (list[list]):    날짜 필드를 커스터마이징 하기 위한 정보 객체 모음
        :param logger (Logger):     로그를 출력할 Logger 객체
        """

        self.csv_file_name = csv_file_name
        self.gen_row_len = gen_row_len
        self.unique_field_count = unique_field_count
        self.date_fields = date_fields
        self.logger = logger

    def get_csv_file_name(self):
        return self.csv_file_name

    def set_csv_file_name(self, csv_file_name):
        self.csv_file_name = csv_file_name

    def get_gen_row_len(self):
        return self.gen_row_len

    def set_gen_row_len(self, gen_row_len):
        self.gen_row_len = gen_row_len

    def get_unique_field_count(self):
        return self.unique_field_count

    def set_unique_field_count(self, unique_field_count=1000):
        self.unique_field_count = unique_field_count
        
    def get_date_fileds(self):
        return self.date_fields

    def set_date_fields(self, date_fields):
        """
        주어진 csv 중 날짜 필드를 커스터마이징 하기 위한 정보 객체 모음 설정

        :param date_fields: [필드명, 시작일, 종료일, 입력 날짜 포맷, 출력 날짜 포맷]
        :code

            # Pregnancies 필드는 2019-01-01 에서 2019-12-31 까지 랜덤하게 생성하되 YYYY-MM-DD 의 포맷을 입력 받고 YYYYMMDD 포맷으로 변환하라
            # Glucose 필드는 2019-01-01 에서 2019-12-31 까지 랜덤하게 생성하되 YYYY-MM 의 포맷을 입력 받고 YYYYMM 포맷으로 변환하라
            date_fields = [
                ['Pregnancies', '2019-01-01', '2019-12-31', '%Y-%m-%d', '%Y%m%d']
                , [' Glucose', '2019-01', '2019-12', '%Y-%m', '%Y%m']
            ]

            dg.set_date_fields(date_fields)
        """
        self.date_fields = date_fields

    def get_logger(self):
        return self.logger

    def set_logger(self, logger):
        self.logger = logger
        

In [5]:

class DLDummyGenerator(DLDummyGenerator):
    
    def _gen_src_meta_info(self, fgen, dataset):
        """
        컬럼별 메타 정보를 소스 코드의 주석으로 저장

        :param fgen (File):     소스 코드 기록을 위한 파일 객체
        :param dataset (DataFrame): 원본 csv 파일의 Pandas DataFrame
        """

        fgen.write('\'\'\'\n')
        fgen.write(self.csv_file_name + ' 파일의 컬럼 별 데이터 한계 값\n')

        column_names = ','.join(dataset.columns)
        fgen.write('Compute,' + column_names + '\n')

        columns_list = []

        for idx in range(len(dataset.columns)):

            column = dataset.columns[idx]
            columns_desc = []

            if dataset[column].dtypes == object or dataset[column].dtypes == str:
                columns_desc.append('')  # min
                columns_desc.append('')  # max
                columns_desc.append('')  # mean
                columns_desc.append('')  # median
                columns_desc.append('')  # null
            else:
                columns_desc.append(dataset[column].min())
                columns_desc.append(dataset[column].max())
                columns_desc.append(dataset[column].mean())
                columns_desc.append(dataset[column].median())
                columns_desc.append(dataset[column].isnull().sum())

            columns_list.append(columns_desc)

        # 컬럼별 설명
        row_descs = ['min()', 'max()', 'mean()', 'median()', 'null count()']

        for idx in range(len(row_descs)):
            column_desc = [i[idx] for i in columns_list]
            fgen.write(row_descs[idx] + ',' + ','.join(np.asarray(column_desc, dtype=np.str, order='C')))
            fgen.write('\n')

        fgen.write('\'\'\'\n\n\n')


In [6]:

class DLDummyGenerator(DLDummyGenerator):
    
    def _gen_src_data_info(self, fgen, dataset):
        """
        딥러닝용 더미 정보를 생성하는 소스 코드를 저장

        :param fgen (File):     소스 코드 기록을 위한 파일 객체
        :param dataset (DataFrame): 원본 csv 파일의 Pandas DataFrame
        """

        # 컬럼별로 랜덤 데이터 생성 코드 작성
        for idx in range(len(dataset.columns)):

            # print(''.join(column.split()) + ', ' + str(dataset[column].dtypes))

            column = dataset.columns[idx]

            # 컬럼명 주석 처리
            fgen.write('# ' + column + '\n')

            date_fields = [item[0] for item in self.date_fields]
            if column in date_fields:
                for date_field in self.date_fields:
                    if date_field[0] == column:
                        fgen.write('gen_df[\"' + column + '\"] = [fake.date_between(\n')
                        fgen.write(
                            '    start_date=datetime.strptime(\'' + date_field[1] + '\', \'' + date_field[3] + '\')\n')
                        fgen.write('    , end_date=datetime.strptime(\'' + date_field[2] + '\', \'' + date_field[
                            3] + '\')).strftime(\'' + date_field[4] + '\')\n')
                        fgen.write('    for _ in range(GEN_ROW_MAX)]\n\n')

            elif dataset[column].dtypes == object or dataset[column].dtypes == str:

                unique_data = dataset[column].unique()
                unique_data_count = len(unique_data)

                if unique_data_count < self.unique_field_count:
                    # 유니크한 문자열 데이터의 수가 특정 갯수보다 작으면 카테고리 형 데이터로 인지한다
                    fgen.write('gen_df[\"' + column + '\"] = choice([\"')
                    fgen.write('\", \"'.join(np.asarray(dataset[column].unique(), dtype=np.str, order='C')))
                    fgen.write('\"], GEN_ROW_MAX)\n\n')

                else:
                    # 일반 문자열 필드
                    fgen.write('gen_df[\"' + column + '\"] = choice(')
                    fgen.write('[fake.word() for _ in range(GEN_ROW_MAX)]')
                    fgen.write(', GEN_ROW_MAX)\n\n')

            else:
                min_value = dataset[column].min()
                max_value = dataset[column].max()

                if ('int' in str(type(min_value))) and ('int' in str(type(max_value))):
                    # 정수형 필드 (min, max 값으로 랜덤 생성)
                    fgen.write(
                        'gen_df[\"' + column + '\"] = random.randint(' +
                        str(min_value) + ', ' + str(max_value) + ' + 1, GEN_ROW_MAX, dtype=\"int64\")\n\n')
                else:
                    fgen.write(
                        'gen_df[\"' + column + '\"] = random.uniform(' +
                        str(min_value) + ', ' + str(max_value) + ', GEN_ROW_MAX)\n\n')


In [7]:

class DLDummyGenerator(DLDummyGenerator):
    
    def gen_src_from_csv(self):
        """
        딥러닝용 더미 데이터 생성 자동화 구현체
        (주어진 csv 파일을 본떠서 더미 csv 파일을 만드는 소스를 생성한다)

        생성할 소스 파일은 이 csv 데이터 파일의 스키마를 재구조화하고 데이터의 min() / max() 값들로 더미 데이터를 만드는 작업을 수행한다
        """

        logger = self.logger

        # 원본 csv 파일 읽기
        dataset = pd.read_csv(self.csv_file_name, encoding='euc-kr')
        logger.d(dataset)

        # 소스 파일 만들기 시작
        gen_src_file_name, _ = os.path.splitext(self.csv_file_name)
        gen_src_file_name = 'gen_' + gen_src_file_name + '.py'
        fgen = open(gen_src_file_name, 'w', encoding='utf-8')

        fgen.write('# 딥러닝용 더미 데이터 생성 소스로 생성됨\n')
        fgen.write('# 생성일 : ' + str(datetime.now()))
        fgen.write('\n\n')
        fgen.write('# ' + gen_src_file_name + '\n\n')
        fgen.write('# gen_' + self.csv_file_name + '\n')
        fgen.write('\n\n')

        # 컬럼별 메타 정보를 소스 코드의 주석으로 저장
        self._gen_src_meta_info(fgen, dataset)

        fgen.write('import pandas as pd\n')
        fgen.write('\n')
        fgen.write('import numpy as np\n')
        fgen.write('from numpy import random\n')
        fgen.write('from numpy.random import choice\n')
        fgen.write('\n')
        fgen.write('\n# https://minus31.github.io/2018/07/28/python-date/\n')
        fgen.write('from datetime import datetime\n')
        fgen.write('\n')
        fgen.write('\n# https://faker.readthedocs.io\n')
        fgen.write('from faker import Faker\n')
        fgen.write('fake = Faker()\n')
        fgen.write('\n')
        fgen.write('\n')
        fgen.write('GEN_ROW_MAX = ' + str(self.gen_row_len) + '       # 생성 할 샘플의 최대 갯수\n')
        fgen.write('\n')
        fgen.write('\n')

        # pd.set_option('display.max_columns', 999)
        # fgen.write('gen_df = pd.DataFrame({\"gen_id\": range(1, GEN_ROW_MAX + 1)})\n')
        fgen.write('gen_df = pd.DataFrame()\n\n')

        # 딥러닝용 더미 정보를 생성하는 소스 코드를 저장
        self._gen_src_data_info(fgen, dataset)

        fgen.write('\ngen_df.to_csv(\'gen_' + self.csv_file_name + '\', index=False)\n')
        fgen.write('\nprint(\'\\ngen_' + self.csv_file_name + ' File Created...\\n\')\n\n')
        fgen.close()

        print('\n' + gen_src_file_name + ' File Created...\n')
        

In [8]:

# 생성할 소스 파일의 기준이 되는 csv 데이터 파일
CSV_FILE_NAME = "pima-indians-diabetes.csv"

# 생성 할 샘플의 최대 갯수
GEN_ROW_MAX = 10

# 유니크한 문자열 (예를 들어서 코드 값) 필드로 뽑아낼 기준의 크기
UNIQUE_FIELD_COUNT = 1000

# 랜덤 날짜를 만들기 위한 정의
# [필드명, 시작일, 종료일, 입력 날짜 포맷, 출력 날짜 포맷]
DATE_FIELDS = [
    ['Pregnancies', '2019-01-01', '2019-12-31', '%Y-%m-%d', '%Y%m%d']
    , [' Glucose', '2019-01', '2019-12', '%Y-%m', '%Y%m']
]


In [9]:

dg = DLDummyGenerator(CSV_FILE_NAME, GEN_ROW_MAX, UNIQUE_FIELD_COUNT, DATE_FIELDS)

# 더미 데이터를 생성하는 파이썬 소스 코드 생성
dg.gen_src_from_csv()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1



gen_pima-indians-diabetes.py File Created...

